# Keyword Detection on Websites
Source: https://platform.stratascratch.com/data-projects/keyword-detection-websites


In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import chardet
from transformers import pipeline

### (1) Load and examine data

In [2]:
traindt = pd.read_csv("./datasets/train.csv")
testdt = pd.read_csv("./datasets/test.csv")

In [3]:
htmldt = []

for i in range(147):
    with open(f"./datasets/htmls/{i}.html", "rb") as f:
        initread = chardet.detect(f.read())
        encoding = initread["encoding"]

    with open(f"./datasets/htmls/{i}.html", "r", encoding=encoding) as f:
        html = f.read()

    soup = BeautifulSoup(html, "html.parser")
    body = soup.get_text(" ", strip=True)
    htmldt.append({"id": i, "html": body})

htmldt = pd.DataFrame(htmldt)

c:\Users\ZHIWEI.WANG\AppData\Local\Programs\Python\Python312\Lib\html\parser.py:171: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  k = self.parse_starttag(i)


In [4]:
htmldt

,id,html
0,0,"Bauchspeicheldrüse | Klinik für Allgemein-, Vi..."
1,1,Elbe-Elster Klinikum - Chirurgie Finsterwalde ...
2,2,Chirurgie der Bauchspeicheldrüse (Pankreaschir...
3,3,Onkologisches Zentrum - Klinikum Bayreuth Aktu...
4,4,Zentrum - Sozialpädiatrisches Zentrum - Städti...
...,...,...
142,142,Tumorboard des Lungenkrebszentrums Möglicherwe...
143,143,Oberarzt (m/w/d) für die HNO-Klinik A- A A+ No...
144,144,Für Ärzte | Vivantes JavaScript scheint in Ihr...
145,145,"Innere Medizin – Hämatologie, Onkologie und Pa..."


### (2) LLM based Classification

In [6]:
pipe = pipeline(
    "question-answering", model="deutsche-telekom/bert-multi-english-german-squad2"
)

model.safetensors:  25%|##5       | 178M/709M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/264 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

c:\Users\ZHIWEI.WANG\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
QA_Input = {
    "context": "Manuel Romero has been working hardly in the repository hugginface/transformers lately",
    "question": "what is the sentiment of Manuel Romero contributions?",
}
res = pipe(QA_Input)
res


{'score': 0.27123144268989563, 'start': 31, 'end': 37, 'answer': 'hardly'}